# IBM Applied D.S Capstone Project
## By Rohit Kumar Singh
## Week 3 Part 3

Explore and cluster the neighborhoods in Toronto.

## 1. Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd
import folium
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
!conda install -c conda-forge lxml --yes
import requests
import lxml.html as lh
from sklearn.cluster import KMeans
print("Libraries imported.")

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


## 2. Scrap data from Wikipedia page into a DataFrame (Based on Previous Part)

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')
r = requests.get(url)

#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

size_of_col = 3
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=size_of_col:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
        
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df = df.replace(r'\n','', regex=True) 
df.columns = ['PostalCode', 'Borough', 'Neighborhood']
df.drop(df.tail(1).index,inplace=True)

df = df[df.Borough != "Not assigned"].reset_index(drop=True)

df = df.replace(r'/',', ', regex=True) 

for index, row in df.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
df.head()

1:"Postal code
"
2:"Borough
"
3:"Neighborhood
"


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


## 3. Defining the new Dataframe Format

In [3]:
df["Latitude"] = ""
df["Longitude"] = ""
df.shape

(103, 5)

In [4]:
# Cleaning the neighborhood with multiple values ( Selecting the first one)
df["Neighborhood"] = df["Neighborhood"].str.split(",", n = 1, expand = True) 
df["Neighborhood"] = df["Neighborhood"].str.split("-", n = 1, expand = True) 
df["Neighborhood"].head(5)

0           Parkwoods
1    Victoria Village
2        Regent Park 
3     Lawrence Manor 
4       Queen's Park 
Name: Neighborhood, dtype: object

In [5]:
df.head(50)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,,
1,M4A,North York,Victoria Village,,
2,M5A,Downtown Toronto,Regent Park,,
3,M6A,North York,Lawrence Manor,,
4,M7A,Downtown Toronto,Queen's Park,,
5,M9A,Etobicoke,Islington Avenue,,
6,M1B,Scarborough,Malvern,,
7,M3B,North York,Don Mills,,
8,M4B,East York,Parkview Hill,,
9,M5B,Downtown Toronto,Garden District,,


## 4. To get coordinates and populate the df 

In [6]:
df1 = df.loc[0:25]
df2 = df.loc[25:50]
df3 = df.loc[51:75]
df4 = df.loc[76:102]

In [7]:
# Need to drop those Neighborhood that the geocode does not find
to_drop_unknown = []
geolocator = Nominatim(user_agent="ny_explorer")
for index, row in df1.iterrows():
    address = row['Neighborhood'] + ', Toronto'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df1.loc[index, 'Latitude'] = latitude
        df1.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

The geograpical coordinate of Parkwoods, Toronto are 43.7587999, -79.3201966.


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

The geograpical coordinate of Victoria Village, Toronto are 43.732658, -79.3111892.
The geograpical coordinate of Regent Park , Toronto are 43.6607056, -79.3604569.
The geograpical coordinate of Lawrence Manor , Toronto are 43.7220788, -79.4375067.
The geograpical coordinate of Queen's Park , Toronto are 43.659659, -79.3903399.
The geograpical coordinate of Islington Avenue, Toronto are 43.6225748, -79.5142154.
The geograpical coordinate of Malvern , Toronto are 43.8091955, -79.2217008.
The geograpical coordinate of Don Mills, Toronto are 43.775347, -79.3459439.
The geograpical coordinate of Parkview Hill , Toronto are 43.7062977, -79.3219073.
The geograpical coordinate of Garden District, Toronto are 43.6564995, -79.3771141.
The geograpical coordinate of Glencairn, Toronto are 43.7087117, -79.4406853.
The geograpical coordinate of West Deane Park , Toronto are 43.6631995, -79.5685684.
The geograpical coordinate of Rouge Hill , Toronto are 43.7802711, -79.1304992.
The geograpical coord

In [9]:
geolocator = Nominatim(user_agent="ny_explorer2")
for index, row in df2.iterrows():
    address = row['Neighborhood'] + ', Toronto'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df2.loc[index, 'Latitude'] = latitude
        df2.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

The geograpical coordinate of Christie, Toronto are 43.6641106, -79.4184051.


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


The geograpical coordinate of Cedarbrae, Toronto are 43.75646655, -79.22669244258802.
The geograpical coordinate of Hillcrest Village, Toronto are 43.7996637, -79.3650189.
The geograpical coordinate of Bathurst Manor , Toronto are 43.76389295, -79.45636693710946.
The geograpical coordinate of Thorncliffe Park, Toronto are 43.704553, -79.3454074.
The geograpical coordinate of Richmond , Toronto are 43.6485875, -79.3913729.
The geograpical coordinate of Dufferin , Toronto are 43.6602019, -79.4357191.
The geograpical coordinate of Scarborough Village, Toronto are 43.7437422, -79.2116324.
The geograpical coordinate of Fairview , Toronto are 43.777758500000004, -79.34429375180316.
The geograpical coordinate of Northwood Park , Toronto are 43.7541351, -79.50448.
The geograpical coordinate of East Toronto, Toronto are 43.6534817, -79.3839347.
The geograpical coordinate of Harbourfront East , Toronto are 43.6400801, -79.3801495.
The geograpical coordinate of Little Portugal , Toronto are 43.64

In [10]:
geolocator = Nominatim(user_agent="ny_explorer3")
for index, row in df3.iterrows():
    address = row['Neighborhood'] + ', Toronto'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df3.loc[index, 'Latitude'] = latitude
        df3.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

The geograpical coordinate of Cliffside , Toronto are 43.7111699, -79.2481769.


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


The geograpical coordinate of Willowdale , Toronto are 43.7615095, -79.4109234.
The geograpical coordinate of Downsview, Toronto are 43.7492988, -79.462248.
The geograpical coordinate of Studio District, Toronto are 43.64958515, -79.39068322559326.
The geograpical coordinate of Bedford Park , Toronto are 43.7373876, -79.4109253.
Cannot do: Del Ray , Toronto, will drop index: 56
The geograpical coordinate of Humberlea , Toronto are 43.7213166, -79.5331605.
The geograpical coordinate of Birch Cliff , Toronto are 43.702112150000005, -79.26009053015676.
The geograpical coordinate of Willowdale, Toronto are 43.7615095, -79.4109234.
The geograpical coordinate of Downsview, Toronto are 43.7492988, -79.462248.
The geograpical coordinate of Lawrence Park, Toronto are 43.729199, -79.4032525.
The geograpical coordinate of Roselawn, Toronto are 43.6994563, -79.4546164.
The geograpical coordinate of Runnymede , Toronto are 43.6517026, -79.4759978.
The geograpical coordinate of Weston, Toronto are 4

In [12]:
geolocator = Nominatim(user_agent="ny_explorer4")
for index, row in df4.iterrows():
    address = row['Neighborhood'] + ', Toronto'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df4.loc[index, 'Latitude'] = latitude
        df4.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

Cannot do: Canada Post Gateway Processing Centre, Toronto, will drop index: 76
The geograpical coordinate of Kingsview Village , Toronto are 43.6995391, -79.5563459.


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


The geograpical coordinate of Agincourt, Toronto are 43.7853531, -79.2785494.
The geograpical coordinate of Davisville, Toronto are 43.697936, -79.3972908.
The geograpical coordinate of University of Toronto , Toronto are 43.663461999999996, -79.39775965337452.
The geograpical coordinate of Runnymede , Toronto are 43.6517026, -79.4759978.
The geograpical coordinate of Clarks Corners , Toronto are 43.7964095, -79.2977951.
The geograpical coordinate of Moore Park , Toronto are 43.6903876, -79.3832965.
The geograpical coordinate of Kensington Market , Toronto are 43.6552136, -79.4022604.
The geograpical coordinate of Milliken , Toronto are 43.8231743, -79.3017626.
The geograpical coordinate of Summerhill West , Toronto are 43.6816776, -79.3905037.
The geograpical coordinate of CN Tower , Toronto are 43.6425637, -79.38708718320467.
The geograpical coordinate of New Toronto , Toronto are 43.6007625, -79.505264.
The geograpical coordinate of South Steeles , Toronto are 43.8161778, -79.314537

In [13]:
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)

(26, 5)
(26, 5)
(25, 5)
(27, 5)


In [14]:
df = df1.append(df2, ignore_index = True)
df = df.append(df3, ignore_index = True)
df = df.append(df4, ignore_index = True)

In [15]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7588,-79.3202
1,M4A,North York,Victoria Village,43.7327,-79.3112
2,M5A,Downtown Toronto,Regent Park,43.6607,-79.3605
3,M6A,North York,Lawrence Manor,43.7221,-79.4375
4,M7A,Downtown Toronto,Queen's Park,43.6597,-79.3903


## 5. Cleaning the dataframe

In [16]:
clean_df = df.drop(to_drop_unknown)
clean_df['Latitude'].replace('', np.nan, inplace=True)
clean_df.dropna(subset=['Latitude'], inplace=True)
clean_df.shape

(96, 5)

In [17]:
clean_df.tail()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
97,M4X,Downtown Toronto,St. James Town,43.669403,-79.3727
98,M5X,Downtown Toronto,First Canadian Place,43.648768,-79.3817
99,M8X,Etobicoke,The Kingsway,43.647381,-79.5113
102,M8Y,Etobicoke,Old Mill South,43.649826,-79.4943
103,M8Z,Etobicoke,Mimico NW,43.616677,-79.4968


## 6. Filter only boroughs that contain the word Toronto

In [18]:
# filter borough names that contain the word Toronto
Toronto_df = clean_df[clean_df['Borough'].str.contains("Toronto")].reset_index(drop=True)
Toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park,43.660706,-79.3605
1,M7A,Downtown Toronto,Queen's Park,43.659659,-79.3903
2,M5B,Downtown Toronto,Garden District,43.656500,-79.3771
3,M5C,Downtown Toronto,St. James Town,43.669403,-79.3727
4,M4E,East Toronto,The Beaches,43.671024,-79.2967


In [19]:
Toronto_df.drop_duplicates(subset ="PostalCode",keep = "first", inplace = True)
Toronto_df.reset_index(drop=True) 
print(Toronto_df.shape)

(34, 5)


## 7. Define Foursquare Credentials and Version

In [20]:
# define Foursquare Credentials and Version
CLIENT_ID = 'NT2C1SLVDXCSGCZPWP3D0JM41SNOZMB5F4IYCL5CR2V20ZIP' # your Foursquare ID
CLIENT_SECRET = 'PS1QM5EW1EVJPXYJK4JF45GJHMHZJBCDZAD4CXQJUMIDC4NM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
toronto_df_new = Toronto_df.copy()

Your credentails:
CLIENT_ID: NT2C1SLVDXCSGCZPWP3D0JM41SNOZMB5F4IYCL5CR2V20ZIP
CLIENT_SECRET:PS1QM5EW1EVJPXYJK4JF45GJHMHZJBCDZAD4CXQJUMIDC4NM


## 8. Top 100 venues that are within a radius of 500 meters.

In [21]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['PostalCode'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,        CLIENT_SECRET,        VERSION,        lat,        long,        radius,         LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post,             borough,            neighborhood,            lat,             long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [22]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1821, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457,Regent Park Aquatic Centre,43.660600,-79.361392,Pool
1,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457,Daniels Spectrum,43.660137,-79.361808,Performing Arts Venue
2,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457,Sumach Espresso,43.658135,-79.359515,Coffee Shop
3,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457,Thai To Go,43.663418,-79.360710,Thai Restaurant
4,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457,Paintbox Bistro,43.660050,-79.362855,Restaurant


### How many venues were returned for each PostalCode

In [23]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count().head()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,45,45,45,45,45,45
M4K,East Toronto,The Danforth West,30,30,30,30,30,30
M4L,East Toronto,India Bazaar,37,37,37,37,37,37
M4M,East Toronto,Studio District,100,100,100,100,100,100
M4N,Central Toronto,Lawrence Park,54,54,54,54,54,54


In [24]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 232 uniques categories.


In [25]:
venues_df['VenueCategory'].unique()[:10]

array(['Pool', 'Performing Arts Venue', 'Coffee Shop', 'Thai Restaurant',
       'Restaurant', 'Pub', 'Animal Shelter', 'Auto Dealership', 'Park',
       'Sushi Restaurant'], dtype=object)

## 9. Analyze Each Area

In [26]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1821, 235)


,PostalCode,Borough,Neighborhoods,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,...,Tree,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M5A,Downtown Toronto,Regent Park,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,Downtown Toronto,Regent Park,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,Downtown Toronto,Regent Park,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,Downtown Toronto,Regent Park,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,Downtown Toronto,Regent Park,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [27]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped.head()

(34, 235)


,PostalCode,Borough,Neighborhoods,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,...,Tree,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,...,0.022222,0.00,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.00
1,M4K,East Toronto,The Danforth West,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.00,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.00
2,M4L,East Toronto,India Bazaar,0.0,0.00,0.0,0.0,0.0,0.0,0.027027,...,0.000000,0.00,0.027027,0.00,0.0,0.0,0.0,0.0,0.0,0.00
3,M4M,East Toronto,Studio District,0.0,0.01,0.0,0.0,0.0,0.0,0.020000,...,0.000000,0.01,0.030000,0.01,0.0,0.0,0.0,0.0,0.0,0.02
4,M4N,Central Toronto,Lawrence Park,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.00,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.00


### Create the new dataframe and display the top 10 venues for each PostalCode.

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(34, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Beach,Thai Restaurant,Japanese Restaurant,Coffee Shop,Park,Bar,Tea Room,Bakery,Breakfast Spot,Pub
1,M4K,East Toronto,The Danforth West,Coffee Shop,Baseball Field,Skating Rink,Grocery Store,Bus Line,Sandwich Place,Fried Chicken Joint,French Restaurant,Market,Metro Station
2,M4L,East Toronto,India Bazaar,Indian Restaurant,Grocery Store,Café,Convenience Store,Bar,Pakistani Restaurant,Diner,Pizza Place,Egyptian Restaurant,Bus Line
3,M4M,East Toronto,Studio District,Coffee Shop,Japanese Restaurant,Cosmetics Shop,Pizza Place,Vegetarian / Vegan Restaurant,Café,Yoga Studio,Sushi Restaurant,Clothing Store,Bar
4,M4N,Central Toronto,Lawrence Park,Italian Restaurant,Sushi Restaurant,Coffee Shop,Bakery,Ice Cream Shop,Pub,Fast Food Restaurant,Asian Restaurant,Bank,Shopping Mall


## 10. Cluster Neighborhoods.
Run k-means to cluster the neighborhood into 5 clusters.

In [29]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 4, 2, 2, 2, 2, 2, 2, 0, 2], dtype=int32)

### Checking shape of dataframe

In [30]:
print(toronto_grouped.shape)
print(neighborhoods_venues_sorted.shape)
print(toronto_df_new.shape)
print(kmeans.labels_.size)

(34, 235)
(34, 13)
(34, 5)
34


In [31]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = toronto_df_new.copy()
# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")
toronto_merged.rename(columns = {'BoroughLatitude':'Latitude', 'BoroughLongitude':'Longitude'}, inplace = True) 
print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(34, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Regent Park,43.660706,-79.3605,2,Coffee Shop,Thai Restaurant,Fast Food Restaurant,Electronics Store,Park,Food Truck,Restaurant,Beer Store,Sushi Restaurant,Pub
1,M7A,Downtown Toronto,Queen's Park,43.659659,-79.3903,4,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Bubble Tea Shop,Restaurant,Japanese Restaurant,Chinese Restaurant,French Restaurant,Ice Cream Shop
2,M5B,Downtown Toronto,Garden District,43.656500,-79.3771,2,Clothing Store,Coffee Shop,Restaurant,Hotel,Fast Food Restaurant,Café,Lingerie Store,Sandwich Place,Tea Room,Thai Restaurant
3,M5C,Downtown Toronto,St. James Town,43.669403,-79.3727,2,Coffee Shop,Pizza Place,Café,Grocery Store,Caribbean Restaurant,Bus Stop,Food & Drink Shop,Restaurant,Market,Italian Restaurant
4,M4E,East Toronto,The Beaches,43.671024,-79.2967,2,Beach,Thai Restaurant,Japanese Restaurant,Coffee Shop,Park,Bar,Tea Room,Bakery,Breakfast Spot,Pub


In [32]:
# sort the results by Cluster Labels
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged.head()

(34, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,M5H,Downtown Toronto,Richmond,43.648587,-79.3914,0,Coffee Shop,Restaurant,Vegetarian / Vegan Restaurant,Café,Thai Restaurant,Hotel,Bar,Speakeasy,Sushi Restaurant,Bookstore
21,M4P,Central Toronto,Davisville North,43.697936,-79.3973,1,Coffee Shop,Italian Restaurant,Sushi Restaurant,Trail,Convenience Store,Pub,Park,Café,Irish Pub,Sandwich Place
0,M5A,Downtown Toronto,Regent Park,43.660706,-79.3605,2,Coffee Shop,Thai Restaurant,Fast Food Restaurant,Electronics Store,Park,Food Truck,Restaurant,Beer Store,Sushi Restaurant,Pub
31,M4V,Central Toronto,Summerhill West,43.681678,-79.3905,2,Italian Restaurant,Sushi Restaurant,Café,Spa,Boutique,Coffee Shop,Sporting Goods Shop,Beer Store,Juice Bar,Kids Store
30,M5T,Downtown Toronto,Kensington Market,43.655214,-79.4023,2,Café,Bar,Coffee Shop,Vegetarian / Vegan Restaurant,Mexican Restaurant,Dessert Shop,Vietnamese Restaurant,Grocery Store,Taco Place,Breakfast Spot


## 11. Plotting the final neighborhood of Toronto Map

In [ ]:
#n = 33  # Unable to show Marker in Map if Marker > 10
#clean_df1 = toronto_merged.tail(n)
#clean_df1.shape

In [33]:
map_clusters  = folium.Map(location=[latitude, longitude], zoom_start=12)
# set color scheme for the clusters
x = np.arange(kclusters)
rainbow = [    'red',    'blue',    'orange',    'darkgreen',    'darkblue',    'black']
# add markers to map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 12. Examine Clusters
### Cluster 0

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

### Cluster 1

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,1,Coffee Shop,Italian Restaurant,Sushi Restaurant,Trail,Convenience Store,Pub,Park,Café,Irish Pub,Sandwich Place


### Cluster 2

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,2,Coffee Shop,Thai Restaurant,Fast Food Restaurant,Electronics Store,Park,Food Truck,Restaurant,Beer Store,Sushi Restaurant,Pub
31,Central Toronto,2,Italian Restaurant,Sushi Restaurant,Café,Spa,Boutique,Coffee Shop,Sporting Goods Shop,Beer Store,Juice Bar,Kids Store
30,Downtown Toronto,2,Café,Bar,Coffee Shop,Vegetarian / Vegan Restaurant,Mexican Restaurant,Dessert Shop,Vietnamese Restaurant,Grocery Store,Taco Place,Breakfast Spot
29,Central Toronto,2,Convenience Store,Playground,Gym,Restaurant,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
28,West Toronto,2,Café,Coffee Shop,Bakery,Bank,Pizza Place,Italian Restaurant,Pharmacy,Pilates Studio,Spa,Flower Shop
27,Downtown Toronto,2,Café,Japanese Restaurant,Park,Bookstore,Italian Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Restaurant,French Restaurant
26,Central Toronto,2,Coffee Shop,Italian Restaurant,Sushi Restaurant,Trail,Convenience Store,Pub,Park,Café,Irish Pub,Sandwich Place
25,Central Toronto,2,Pizza Place,Fried Chicken Joint,Park,Coffee Shop,Thai Restaurant,Bistro,Indian Restaurant,Ice Cream Shop,Donut Shop,Fast Food Restaurant
24,Central Toronto,2,Clothing Store,Coffee Shop,American Restaurant,Restaurant,Diner,Breakfast Spot,Hotel,Steakhouse,Plaza,Theater
23,West Toronto,2,Convenience Store,Pet Store,Deli / Bodega,Tennis Court,Mexican Restaurant,Sandwich Place,Pub,Pool,Gym,Pizza Place


### Cluster 3

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Downtown Toronto,3,Hotel,Pizza Place,Coffee Shop,Bar,Ice Cream Shop,Aquarium,Baseball Stadium,Gym,Scenic Lookout,Monument / Landmark


### Cluster 4

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Downtown Toronto,4,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Pizza Place,Steakhouse,Sports Bar,History Museum,Sandwich Place
12,West Toronto,4,Bar,Café,Coffee Shop,Restaurant,Korean Restaurant,Cocktail Bar,Bakery,Health & Beauty Service,Dive Bar,Breakfast Spot
14,Downtown Toronto,4,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Japanese Restaurant,Seafood Restaurant,Beer Bar,Concert Hall,Tea Room
1,Downtown Toronto,4,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Bubble Tea Shop,Restaurant,Japanese Restaurant,Chinese Restaurant,French Restaurant,Ice Cream Shop
34,Downtown Toronto,4,Coffee Shop,Restaurant,Hotel,Café,American Restaurant,Gym,Japanese Restaurant,Asian Restaurant,Seafood Restaurant,Deli / Bodega


## 13. Observations:
Most of the neighborhoods fall into Cluster 2 which are mostly business areas with cafe, restaurants, supermarkets etc. Cluster 0 & Cluster 1 has most common single area of coffee shop, Cluster 3 has one most common venue as hotel, and lastly Cluster 4 has coffee shop and bars trail.